In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

import tensorflow as tf
import tensorflow_hub as hub
from absl import logging

In [ ]:
tf.__version__

# Datasets

In [ ]:
dictionary = pd.read_csv("../processed_data/dictionary.csv").drop(columns=['Unnamed: 0'])
text_by_paragraph = pd.read_csv("../processed_data/text_by_paragraph.csv").drop(columns=['Unnamed: 0'])

# Universal Sentence Encoder (used by google)

In [ ]:
# Put a reference

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [ ]:
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(text_by_paragraph['Corpus'].tolist())
dic_embeddings = embed(dictionary['word'].tolist())

# Calculating the cosine similarity

In [ ]:
import matplotlib.pyplot as plt
cosine_matrix = cosine_similarity(message_embeddings, dic_embeddings)##
plt.hist(cosine_matrix)

In [45]:
#comments, topics = np.where(cosine_matrix>=0.15)
comments = [idx for idx, val in list(enumerate(np.argmax(cosine_matrix, axis=1)))]
topics = [val for idx, val in list(enumerate(np.argmax(cosine_matrix, axis=1)))]

In [46]:
paragraphs = text_by_paragraph.loc[comments, ['Document_name', 'Order']].reset_index().drop(columns=['index'])
topics_df = dictionary.iloc[topics].reset_index().drop(columns=['index'])

topics_references = pd.concat([paragraphs,topics_df], axis=1)

In [ ]:
#unique_topics = topics_references[['Document_name','Order']].drop_duplicates()
#unique_do = text_by_paragraph[['Document_name','Order']].drop_duplicates()

#print(unique_topics.shape)
#print(unique_do.shape)

In [49]:
topics_references.to_csv("../processed_data/topics.csv")